## 01. Import data and libraries

In [1]:
# Import libraries 

import pandas as pd
import numpy as np
import os

In [2]:
# Define path to data

path = r'/Users/louise/Desktop/CF Coursework/Achievement 4 /Instacart Basket Analysis'

In [4]:
# Import prepared data


ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data','orders_products_variables.pkl'))

In [5]:
# Check data

ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy days,Average Orders
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy days,Average Orders
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy days,Most Orders
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy days,Average Orders
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy days,Most Orders


## 02. Task- group and aggregate

In [6]:
# Find the average number of orders for each depatment in the whole dataframe

ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


Notes: All 21 departments are represented in this list, while only 8 of them were represented in the first million rows. The average orders for each department are in the same ballpark in both lists (mostly mid teens to 20). Department 17 was under-represented in the first million rows.

##  03. Create a loyalty flag for the whole dataframe

In [7]:
# Create a column storing the maximum order # for each user_id

ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])f.transform(np.max)

In [8]:
# Create a column distinguishing among the most loyal, least loyal, and average customers

ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [9]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [10]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [11]:
# Check value counts

ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [12]:
# Check new dataframe 

ords_prods_merge[['user_id', 'loyalty_flag', 'max_order']].head(30)

,user_id,loyalty_flag,max_order
0,1,New customer,10
1,1,New customer,10
2,1,New customer,10
3,1,New customer,10
4,1,New customer,10
5,1,New customer,10
6,1,New customer,10
7,1,New customer,10
8,1,New customer,10
9,1,New customer,10


## 04. Investigate spending habits of loyalty groups

In [13]:
# Find the average price of products purchased by each loyalty group

ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean']})

,prices
,mean
loyalty_flag,
Loyal customer,10.386336
New customer,13.294670
Regular customer,12.495717


In [14]:
# Find the median 

ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['median']})

,prices
,median
loyalty_flag,
Loyal customer,7.4
New customer,7.4
Regular customer,7.4


Notes: New customers, who make fewer total orders, buy more expensive products on average than regular customers, regular customers than loyal customers. 

## 05. Create a spending flag

In [15]:
# Create a column of the mean price of products purchased by each user_id

ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [16]:
# Create a column distinguishing high spenders from low spenders

ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spending_flag'] = 'High Spender'

In [17]:
ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spending_flag'] = 'Low Spender'

In [18]:
# Check value counts


ords_prods_merge['spending_flag'].value_counts(dropna = False)

Low Spender     31770614
High Spender      634245
Name: spending_flag, dtype: int64

In [19]:
# Check new dataframe 

ords_prods_merge[['user_id', 'spending_flag', 'avg_price']].head(30)

,user_id,spending_flag,avg_price
0,1,Low Spender,6.367797
1,1,Low Spender,6.367797
2,1,Low Spender,6.367797
3,1,Low Spender,6.367797
4,1,Low Spender,6.367797
5,1,Low Spender,6.367797
6,1,Low Spender,6.367797
7,1,Low Spender,6.367797
8,1,Low Spender,6.367797
9,1,Low Spender,6.367797


## 06. Flag frequent vs. infrequent customers

In [20]:
# Create a column of the mean price of products purchased by each user_id

ords_prods_merge['median_days_since'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [21]:
# Create a column distinguishing frequent from infrequent customers

result = []

for value in ords_prods_merge['median_days_since']:
  if value > 20:
    result.append('Infrequent')
  elif value <= 10:
    result.append('Frequent')
  else:
    result.append('Regular')

In [22]:
ords_prods_merge ['frequency_flag'] = result

In [23]:
# Check value counts

ords_prods_merge ['frequency_flag'].value_counts(dropna = False)

Frequent      21559853
Regular        7208569
Infrequent     3636437
Name: frequency_flag, dtype: int64

In [24]:
# Check new dataframe

ords_prods_merge[['user_id', 'frequency_flag', 'median_days_since']].head(60)

,user_id,frequency_flag,median_days_since
0,1,Infrequent,20.5
1,1,Infrequent,20.5
2,1,Infrequent,20.5
3,1,Infrequent,20.5
4,1,Infrequent,20.5
5,1,Infrequent,20.5
6,1,Infrequent,20.5
7,1,Infrequent,20.5
8,1,Infrequent,20.5
9,1,Infrequent,20.5


## 07. Export new data frame 

In [28]:
# Export and save

ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_grouped.pkl'))